In [1]:
import probtorch
import torch

import combinators
import smc
import ssm
import utils

In [2]:
generative = smc.ParticleTrace(1)

In [3]:
mu = torch.zeros(1, 1)
sigma = torch.ones(1, 1) / 2
delta = torch.ones(1, 1)
zs = torch.ones(1, 50+1) * -1
zs[:, 0] = 0

In [4]:
for t in range(zs.shape[1] - 1):
    zs, mu, sigma, delta, generative = ssm.ssm_step(zs, mu, sigma, delta, t+1, generative)

In [5]:
generative

Trace{'Z_1': Normal([torch.FloatTensor of size 1x1]), 'X_1': Normal([torch.FloatTensor of size 1]), 'Z_2': Normal([torch.FloatTensor of size 1x1]), 'X_2': Normal([torch.FloatTensor of size 1]), 'Z_3': Normal([torch.FloatTensor of size 1x1]), 'X_3': Normal([torch.FloatTensor of size 1]), 'Z_4': Normal([torch.FloatTensor of size 1x1]), 'X_4': Normal([torch.FloatTensor of size 1]), 'Z_5': Normal([torch.FloatTensor of size 1x1]), 'X_5': Normal([torch.FloatTensor of size 1]), 'Z_6': Normal([torch.FloatTensor of size 1x1]), 'X_6': Normal([torch.FloatTensor of size 1]), 'Z_7': Normal([torch.FloatTensor of size 1x1]), 'X_7': Normal([torch.FloatTensor of size 1]), 'Z_8': Normal([torch.FloatTensor of size 1x1]), 'X_8': Normal([torch.FloatTensor of size 1]), 'Z_9': Normal([torch.FloatTensor of size 1x1]), 'X_9': Normal([torch.FloatTensor of size 1]), 'Z_10': Normal([torch.FloatTensor of size 1x1]), 'X_10': Normal([torch.FloatTensor of size 1]), 'Z_11': Normal([torch.FloatTensor of size 1x1]), 'X_

In [6]:
num_particles = 500

In [7]:
smc_step = smc.smc(ssm.ssm_step, ssm.ssm_retrace)

In [8]:
ssm_params = {
    'mu': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    },
    'sigma': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    },
    'delta': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    }
}

In [9]:
inference, init_ssm = smc.variational_smc(num_particles, ssm.init_ssm, smc_step, 500, 50, ssm_params, generative, 5)

In [10]:
inference

Trace{'mu': Normal([torch.FloatTensor of size 500]), 'sigma': Normal([torch.FloatTensor of size 500]), 'delta': Normal([torch.FloatTensor of size 500]), 'Z_0': Normal([torch.FloatTensor of size 500]), 'Z_1': Normal([torch.FloatTensor of size 500]), 'X_1': Normal([torch.FloatTensor of size 1]), 'Z_2': Normal([torch.FloatTensor of size 500]), 'X_2': Normal([torch.FloatTensor of size 1]), 'Z_3': Normal([torch.FloatTensor of size 500]), 'X_3': Normal([torch.FloatTensor of size 1]), 'Z_4': Normal([torch.FloatTensor of size 500]), 'X_4': Normal([torch.FloatTensor of size 1]), 'Z_5': Normal([torch.FloatTensor of size 500]), 'X_5': Normal([torch.FloatTensor of size 1]), 'Z_6': Normal([torch.FloatTensor of size 500]), 'X_6': Normal([torch.FloatTensor of size 1]), 'Z_7': Normal([torch.FloatTensor of size 500]), 'X_7': Normal([torch.FloatTensor of size 1]), 'Z_8': Normal([torch.FloatTensor of size 500]), 'X_8': Normal([torch.FloatTensor of size 1]), 'Z_9': Normal([torch.FloatTensor of size 500]),

In [11]:
generative

Trace{'Z_1': Normal([torch.FloatTensor of size 1x1]), 'X_1': Normal([torch.FloatTensor of size 1]), 'Z_2': Normal([torch.FloatTensor of size 1x1]), 'X_2': Normal([torch.FloatTensor of size 1]), 'Z_3': Normal([torch.FloatTensor of size 1x1]), 'X_3': Normal([torch.FloatTensor of size 1]), 'Z_4': Normal([torch.FloatTensor of size 1x1]), 'X_4': Normal([torch.FloatTensor of size 1]), 'Z_5': Normal([torch.FloatTensor of size 1x1]), 'X_5': Normal([torch.FloatTensor of size 1]), 'Z_6': Normal([torch.FloatTensor of size 1x1]), 'X_6': Normal([torch.FloatTensor of size 1]), 'Z_7': Normal([torch.FloatTensor of size 1x1]), 'X_7': Normal([torch.FloatTensor of size 1]), 'Z_8': Normal([torch.FloatTensor of size 1x1]), 'X_8': Normal([torch.FloatTensor of size 1]), 'Z_9': Normal([torch.FloatTensor of size 1x1]), 'X_9': Normal([torch.FloatTensor of size 1]), 'Z_10': Normal([torch.FloatTensor of size 1x1]), 'X_10': Normal([torch.FloatTensor of size 1]), 'Z_11': Normal([torch.FloatTensor of size 1x1]), 'X_

In [12]:
for t in range(1, zs.shape[1]):
    key = 'Z_%d' % t
    accuracy = ((inference[key].value - generative[key].value)**2).mean()
    print('SMC MSE at time %d: %f' % (t, accuracy))

SMC MSE at time 1: 3.191959
SMC MSE at time 2: 3.166860
SMC MSE at time 3: 4.757319
SMC MSE at time 4: 2.898298
SMC MSE at time 5: 25.023756
SMC MSE at time 6: 0.011953
SMC MSE at time 7: 0.002487
SMC MSE at time 8: 0.546920
SMC MSE at time 9: 0.464412
SMC MSE at time 10: 2.925769
SMC MSE at time 11: 0.150761
SMC MSE at time 12: 3.003742
SMC MSE at time 13: 0.357159
SMC MSE at time 14: 30.186310
SMC MSE at time 15: 0.081270
SMC MSE at time 16: 5.875355
SMC MSE at time 17: 4.542355
SMC MSE at time 18: 0.277228
SMC MSE at time 19: 0.002280
SMC MSE at time 20: 19.837828
SMC MSE at time 21: 1.065374
SMC MSE at time 22: 0.057243
SMC MSE at time 23: 39.317284
SMC MSE at time 24: 8.365187
SMC MSE at time 25: 43.978592
SMC MSE at time 26: 0.030107
SMC MSE at time 27: 7.033696
SMC MSE at time 28: 1.191936
SMC MSE at time 29: 3.226419
SMC MSE at time 30: 0.199922
SMC MSE at time 31: 2.426862
SMC MSE at time 32: 3.649410
SMC MSE at time 33: 0.550587
SMC MSE at time 34: 13.104147
SMC MSE at time 3